In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

import statsmodels.api as sm

## Import

In [17]:
wX_train = pd.read_csv("wXtrain.csv")
wX_val = pd.read_csv("wXval.csv")
wX_test = pd.read_csv("wXtest.csv")

wy_train = pd.read_csv("wytrain.csv")
wy_val = pd.read_csv("wyval.csv")
wy_test = pd.read_csv("wytest.csv")

mX_train = pd.read_csv("mXtrain.csv")
mX_val = pd.read_csv("mXval.csv")
mX_test = pd.read_csv("mXtest.csv")

my_train = pd.read_csv("mytrain.csv")
my_val = pd.read_csv("myval.csv")
my_test = pd.read_csv("mytest.csv")

## Scaling

In [18]:
from sklearn.preprocessing import RobustScaler
scaler = StandardScaler()

mX_train[mX_train.columns] = scaler.fit_transform(mX_train[mX_train.columns])
wX_train[wX_train.columns] = scaler.fit_transform(wX_train[wX_train.columns])

mX_val[mX_val.columns] = scaler.fit_transform(mX_val[mX_val.columns])
mX_test[mX_test.columns] = scaler.fit_transform(mX_test[mX_test.columns])

## Logistic Regression 

### Men

In [19]:
traits = sm.Logit(my_train, mX_train)
results_traits = traits.fit()
results_traits.summary()

Optimization terminated successfully.
         Current function value: 0.456369
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    dec   No. Observations:                 2063
Model:                          Logit   Df Residuals:                     2053
Method:                           MLE   Df Model:                            9
Date:                Tue, 25 Jan 2022   Pseudo R-squ.:                  0.3415
Time:                        13:39:00   Log-Likelihood:                -941.49
converged:                       True   LL-Null:                       -1429.8
Covariance Type:            nonrobust   LLR p-value:                1.872e-204
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
attr           1.0252      0.088     11.684      0.000       0.853       1.197
sinc          -0.5181      0.085     -6.092      0.000      -0.685      -0.351
intel         -0.1290      0.089     -1.454      0.146      -0.303       0.045
fun            0.2572      0.087      2.961      0.003       0.087       0.428
like           1.2306      0.107     11.495      0.000       1.021       1.440
int_corr      -0.0754      0.058     -1.307      0.191      -0.188       0.038
amb           -0.2536      0.079     -3.228      0.001      -0.408      -0.100
samerace      -0.0995      0.058     -1.730      0.084      -0.212       0.013
exphappy      -0.1816      0.060     -3.037      0.002      -0.299      -0.064
prob           0.3606      0.072      5.025      0.000       0.220       0.501
==============================================================================
"""

### Women

In [20]:
#sns.heatmap(attr_clean_women.corr(), annot=True)

In [21]:
traits = sm.Logit(wy_train, wX_train)
results_traits = traits.fit()
results_traits.summary()

Optimization terminated successfully.
         Current function value: 0.532809
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    dec   No. Observations:                 2028
Model:                          Logit   Df Residuals:                     2018
Method:                           MLE   Df Model:                            9
Date:                Tue, 25 Jan 2022   Pseudo R-squ.:                  0.1980
Time:                        13:39:02   Log-Likelihood:                -1080.5
converged:                       True   LL-Null:                       -1347.4
Covariance Type:            nonrobust   LLR p-value:                3.477e-109
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
attr           0.6587      0.074      8.889      0.000       0.513       0.804
sinc          -0.1549      0.079     -1.962      0.050      -0.310      -0.000
intel         -0.0160      0.087     -0.185      0.854      -0.186       0.154
fun            0.3731      0.080      4.667      0.000       0.216       0.530
like           0.6236      0.091      6.875      0.000       0.446       0.801
int_corr      -0.0192      0.053     -0.362      0.717      -0.123       0.085
amb           -0.1204      0.075     -1.615      0.106      -0.267       0.026
samerace       0.0306      0.053      0.577      0.564      -0.073       0.135
exphappy       0.2745      0.055      5.032      0.000       0.168       0.381
prob           0.2404      0.061      3.911      0.000       0.120       0.361
==============================================================================
"""

## Comparison of Classifiers

In [7]:
wX_train = pd.read_csv("wXtrain.csv")
wX_val = pd.read_csv("wXval.csv")
wX_test = pd.read_csv("wXtest.csv")

wy_train = pd.read_csv("wytrain.csv")
wy_val = pd.read_csv("wyval.csv")
wy_test = pd.read_csv("wytest.csv")

mX_train = pd.read_csv("mXtrain.csv")
mX_val = pd.read_csv("mXval.csv")
mX_test = pd.read_csv("mXtest.csv")

my_train = pd.read_csv("mytrain.csv")
my_val = pd.read_csv("myval.csv")
my_test = pd.read_csv("mytest.csv")

In [8]:
def train_val_score(X_train, y_train, X_val, y_val, model):
    '''
    Train and Scores
    '''
    model.fit(X_train, y_train)
    y_hat = model.predict(X_val)
    scores = (accuracy_score(y_val, y_hat), 
             precision_score(y_val, y_hat), 
             recall_score(y_val, y_hat))
    return scores

In [9]:
def test_score(X_test,y_test, model):
    '''
    Test Score
    '''
    y_hat = model.predict(X_test)
    scores = (
            accuracy_score(y_test, y_hat), 
             precision_score(y_test, y_hat), 
             recall_score(y_test, y_hat),
             confusion_matrix(y_test, y_hat)
             )
    return scores

## Women

In [10]:
model_lr = LogisticRegression()

model_svc = svm.SVC(kernel='rbf', C=2)

model_nn = MLPClassifier(max_iter=10000, early_stopping=True, learning_rate="adaptive", 
                      hidden_layer_sizes=(100,), learning_rate_init=1e-1)

model_rf = RandomForestClassifier(max_depth=10, n_estimators=10, max_features=6)

model_nb = GaussianNB()

model_nc = KNeighborsClassifier(5)

In [11]:
score_lr = train_val_score(wX_train, wy_train.to_numpy().reshape(wX_train.shape[0]), 
                           wX_val, wy_val.to_numpy().reshape(wX_val.shape[0]),
                            model_lr)

score_svc = train_val_score(wX_train, wy_train.to_numpy().reshape(wX_train.shape[0]), 
                           wX_val, wy_val.to_numpy().reshape(wX_val.shape[0]),
                            model_svc)

score_nn = train_val_score(wX_train, wy_train.to_numpy().reshape(wX_train.shape[0]), 
                           wX_val, wy_val.to_numpy().reshape(wX_val.shape[0]),
                            model_nn)

score_rf = train_val_score(wX_train, wy_train.to_numpy().reshape(wX_train.shape[0]), 
                           wX_val, wy_val.to_numpy().reshape(wX_val.shape[0]),
                            model_rf)

score_nb = train_val_score(wX_train, wy_train.to_numpy().reshape(wX_train.shape[0]), 
                           wX_val, wy_val.to_numpy().reshape(wX_val.shape[0]), 
                            model_nb)

score_nc = train_val_score(wX_train, wy_train.to_numpy().reshape(wX_train.shape[0]), 
                           wX_val, wy_val.to_numpy().reshape(wX_val.shape[0]), 
                            model_nc)

print(f"Score Logistic Regression: Mean Accuracy: {round(score_lr[0],4)}")
print(f"Score Logistic Regression: Mean Precision: {round(score_lr[1],4)}")
print(f"Score Logistic Regression: Mean Recall: {round(score_lr[2],4)}")
print("\n --- \n")

print(f"Score Support Vector Classifier: Mean Accuracy: {round(score_svc[0],4)}")
print(f"Score Support Vector Classifier: Mean Precision: {round(score_svc[1],4)}")
print(f"Score Support Vector Classifier: Mean Recall: {round(score_svc[2],4)}")
print("\n --- \n")

print(f"Score Multi Layer Perceptron (1 Hidden Layer): Mean Accuracy: {round(score_nn[0],4)}")
print(f"Score Multi Layer Perceptron (1 Hidden Layer): Mean Precision:  {round(score_nn[1],4)}")
print(f"Score Multi Layer Perceptron (1 Hidden Layer): Mean Recall: {round(score_nn[2],4)}")
print("\n --- \n")

print(f"Random Forest: Mean Accuracy: {round(score_rf[0],4)}")
print(f"Random Forest: Mean Precision:  {round(score_rf[1],4)}")
print(f"Random Forest: Mean Recall: {round(score_rf[2],4)}")
print("\n --- \n")

print(f"Naive Bayes: Mean Accuracy: {round(score_nb[0],4)}")
print(f"Naive Bayes: Mean Precision: {round(score_nb[1],4)}")
print(f"Naive Bayes: Mean Recall: {round(score_nb[2],4)}")
print("\n --- \n")

print(f"Nearest Neighbor: Mean Accuracy: {round(score_nc[0],4)}")
print(f"Nearest Neighbor: Mean Precision: {round(score_nc[1],4)}")
print(f"Nearest Neighbor: Mean Recall: {round(score_nc[2],4)}")

Score Logistic Regression: Mean Accuracy: 0.7575
Score Logistic Regression: Mean Precision: 0.6787
Score Logistic Regression: Mean Recall: 0.6065

 --- 

Score Support Vector Classifier: Mean Accuracy: 0.7609
Score Support Vector Classifier: Mean Precision: 0.6992
Score Support Vector Classifier: Mean Recall: 0.5774

 --- 

Score Multi Layer Perceptron (1 Hidden Layer): Mean Accuracy: 0.7391
Score Multi Layer Perceptron (1 Hidden Layer): Mean Precision:  0.7173
Score Multi Layer Perceptron (1 Hidden Layer): Mean Recall: 0.4419

 --- 

Random Forest: Mean Accuracy: 0.7299
Random Forest: Mean Precision:  0.6384
Random Forest: Mean Recall: 0.5581

 --- 

Naive Bayes: Mean Accuracy: 0.7
Naive Bayes: Mean Precision: 0.5614
Naive Bayes: Mean Recall: 0.7226

 --- 

Nearest Neighbor: Mean Accuracy: 0.7264
Nearest Neighbor: Mean Precision: 0.6277
Nearest Neighbor: Mean Recall: 0.571


In [12]:
score_test = test_score(wX_test, wy_test.to_numpy().reshape(wX_test.shape[0]),
                            model_lr)

print(f"Score Test: Accuracy: {round(score_test[0],2)}")
print(f"Score Test: Precision: {round(score_test[1],2)}")
print(f"Score Test: Recall: {round(score_test[2],2)}")

print(3*"---")

print(f"Confusion Matrix: {score_test[3]}")

Score Test: Accuracy: 0.75
Score Test: Precision: 0.67
Score Test: Recall: 0.62
---------
Confusion Matrix: [[381  79]
 [101 164]]


## Men

In [13]:
model_lr = LogisticRegression()

model_svc = svm.SVC(kernel='rbf', C=2)

model_nn = MLPClassifier(max_iter=10000, early_stopping=True, learning_rate="adaptive", 
                      hidden_layer_sizes=(100,), learning_rate_init=1e-1)

model_rf = RandomForestClassifier(max_depth=10, n_estimators=10, max_features=6)

model_nb = GaussianNB()

model_nc = KNeighborsClassifier(5)

In [14]:
score_lr = train_val_score(mX_train, my_train.to_numpy().reshape(mX_train.shape[0]), 
                           mX_val, my_val.to_numpy().reshape(mX_val.shape[0]),
                            model_lr)

score_svc = train_val_score(mX_train, my_train.to_numpy().reshape(mX_train.shape[0]), 
                           mX_val, my_val.to_numpy().reshape(mX_val.shape[0]),
                            model_svc)

score_nn = train_val_score(mX_train, my_train.to_numpy().reshape(mX_train.shape[0]), 
                           mX_val, my_val.to_numpy().reshape(mX_val.shape[0]),
                            model_nn)

score_rf = train_val_score(mX_train, my_train.to_numpy().reshape(mX_train.shape[0]), 
                           mX_val, my_val.to_numpy().reshape(mX_val.shape[0]),
                            model_rf)

score_nb = train_val_score(mX_train, my_train.to_numpy().reshape(mX_train.shape[0]), 
                           mX_val, my_val.to_numpy().reshape(mX_val.shape[0]),
                            model_nb)

score_nc = train_val_score(mX_train, my_train.to_numpy().reshape(mX_train.shape[0]), 
                           mX_val, my_val.to_numpy().reshape(mX_val.shape[0]),
                            model_nc)

print(f"Score Logistic Regression: Mean Accuracy: {round(score_lr[0],4)}")
print(f"Score Logistic Regression: Mean Precision: {round(score_lr[1],4)}")
print(f"Score Logistic Regression: Mean Recall: {round(score_lr[2],4)}")
print("\n --- \n")

print(f"Score Support Vector Classifier: Mean Accuracy: {round(score_svc[0],4)}")
print(f"Score Support Vector Classifier: Mean Precision: {round(score_svc[1],4)}")
print(f"Score Support Vector Classifier: Mean Recall: {round(score_svc[2],4)}")
print("\n --- \n")

print(f"Score Multi Layer Perceptron (1 Hidden Layer): Mean Accuracy: {round(score_nn[0],4)}")
print(f"Score Multi Layer Perceptron (1 Hidden Layer): Mean Precision:  {round(score_nn[1],4)}")
print(f"Score Multi Layer Perceptron (1 Hidden Layer): Mean Recall: {round(score_nn[2],4)}")
print("\n --- \n")

print(f"Random Forest: Mean Accuracy: {round(score_rf[0],4)}")
print(f"Random Forest: Mean Precision:  {round(score_rf[1],4)}")
print(f"Random Forest: Mean Recall: {round(score_rf[2],4)}")
print("\n --- \n")

print(f"Naive Bayes: Mean Accuracy: {round(score_nb[0],4)}")
print(f"Naive Bayes: Mean Precision: {round(score_nb[1],4)}")
print(f"Naive Bayes: Mean Recall: {round(score_nb[2],4)}")
print("\n --- \n")

print(f"Nearest Neighbor: Mean Accuracy: {round(score_nc[0],4)}")
print(f"Nearest Neighbor: Mean Precision: {round(score_nc[1],4)}")
print(f"Nearest Neighbor: Mean Recall: {round(score_nc[2],4)}")

Score Logistic Regression: Mean Accuracy: 0.7785
Score Logistic Regression: Mean Precision: 0.7528
Score Logistic Regression: Mean Recall: 0.7957

 --- 

Score Support Vector Classifier: Mean Accuracy: 0.7808
Score Support Vector Classifier: Mean Precision: 0.742
Score Support Vector Classifier: Mean Recall: 0.8266

 --- 

Score Multi Layer Perceptron (1 Hidden Layer): Mean Accuracy: 0.7469
Score Multi Layer Perceptron (1 Hidden Layer): Mean Precision:  0.7039
Score Multi Layer Perceptron (1 Hidden Layer): Mean Recall: 0.8076

 --- 

Random Forest: Mean Accuracy: 0.7672
Random Forest: Mean Precision:  0.7394
Random Forest: Mean Recall: 0.7886

 --- 

Naive Bayes: Mean Accuracy: 0.7571
Naive Bayes: Mean Precision: 0.7137
Naive Bayes: Mean Recall: 0.8171

 --- 

Nearest Neighbor: Mean Accuracy: 0.7571
Nearest Neighbor: Mean Precision: 0.7155
Nearest Neighbor: Mean Recall: 0.8124


In [15]:
score_test = test_score(wX_test, wy_test.to_numpy().reshape(wX_test.shape[0]),
                            model_lr)

print(f"Score Test: Accuracy: {round(score_test[0],2)}")
print(f"Score Test: Precision: {round(score_test[1],2)}")
print(f"Score Test: Recall: {round(score_test[2],2)}")

print(3*"---")

print(f"Confusion Matrix: {score_test[3]}")

Score Test: Accuracy: 0.74
Score Test: Precision: 0.62
Score Test: Recall: 0.75
---------
Confusion Matrix: [[338 122]
 [ 66 199]]
